In [49]:
import numpy as np
import pandas as pd

In [135]:
PATH_CAT_CROSSMATCH = '../Data/COSMOS/Catalog/CrossMatch/'

In [139]:
# Ugne
# df_radio  = pd.read_csv('%scosmos_match_450um_Ugne_Lim_3GHzlp_irac.csv'%(PATH_CAT_CROSSMATCH))
# df_Nradio = pd.read_csv('%scosmos_match_450um_Ugne_Lim_3GHzlp_mips_irac.csv'%(PATH_CAT_CROSSMATCH))
# df_radio  = pd.read_csv('%scosmos_match_450um_Ugne_Lim_1d4GHzXS_3GHzlp_irac.csv'%(PATH_CAT_CROSSMATCH))
# df_Nradio = pd.read_csv('%scosmos_match_450um_Ugne_Lim_1d4GHzXS_3GHzlp_mips_irac.csv'%(PATH_CAT_CROSSMATCH))
# Lim
# df_radio  = pd.read_csv('%scosmos_match_450um_Lim_3GHzlp_irac.csv'%(PATH_CAT_CROSSMATCH))
# df_Nradio = pd.read_csv('%scosmos_match_450um_Lim_3GHzlp_mips_irac.csv'%(PATH_CAT_CROSSMATCH))
# df_radio  = pd.read_csv('%scosmos_match_450um_Lim_1d4GHzXS_3GHzlp_irac.csv'%(PATH_CAT_CROSSMATCH))
# df_Nradio = pd.read_csv('%scosmos_match_450um_Lim_1d4GHzXS_3GHzlp_mips_irac.csv'%(PATH_CAT_CROSSMATCH))
# # Gao
# df_radio  = pd.read_csv('%scosmos_match_450um_Gao_Lim_3GHzlp_irac.csv'%(PATH_CAT_CROSSMATCH))
# df_Nradio = pd.read_csv('%scosmos_match_450um_Gao_Lim_3GHzlp_mips_irac.csv'%(PATH_CAT_CROSSMATCH))
df_radio  = pd.read_csv('%scosmos_match_450um_Gao_Lim_1d4GHzXS_3GHzlp_irac.csv'%(PATH_CAT_CROSSMATCH))
df_Nradio = pd.read_csv('%scosmos_match_450um_Gao_Lim_1d4GHzXS_3GHzlp_mips_irac.csv'%(PATH_CAT_CROSSMATCH))

In [141]:
# label_cat = 'Ugne'
# label_cat = 'Lim'
label_cat = 'Gao'
# snr_thld_lst = [5]
snr_thld_lst = [3.5, 4, 4.5, 5]

# column_S1d4GHz = 'flux_1d4GHz'
# column_radio = 'ra_3GHz'
column_S1d4GHz = 'Flux_corr_1d4GHz'
column_radio = 'ra_radio'
# column_mips = 'RA_mips24_LeFloch'
column_mips = 'RA_mips24_whw'

for snr_thld in snr_thld_lst:
#     df_radio_SNRslt = df_radio
#     df_Nradio_SNRslt = df_Nradio
    df_radio_SNRslt = df_radio[ df_radio['SNR_450']>snr_thld ]
    df_Nradio_SNRslt = df_Nradio[ df_Nradio['SNR_450']>snr_thld ]

    num_all  = df_radio_SNRslt.shape[0]
    num_1d4GHz = df_radio_SNRslt[column_S1d4GHz].notna().sum()
    num_3GHz   = df_radio_SNRslt['flux_3GHz'].notna().sum()
    num_1d4GHz_3GHz = df_radio_SNRslt[ df_radio_SNRslt['flux_3GHz'].notna() & 
                                 df_radio_SNRslt[column_S1d4GHz].notna()].shape[0]

    # radio detected
    try:
        num_radio  = df_radio_SNRslt[column_radio].notna().sum()
    except KeyError:
        df_radio_SNRslt['ra_radio'] = np.where(df_radio_SNRslt['ra_3GHz'].notna(), 
                                               df_radio_SNRslt['ra_3GHz'], df_radio_SNRslt[column_S1d4GHz])
        num_radio  = df_radio_SNRslt[column_radio].notna().sum()
        
    num_radio_mips  = df_radio_SNRslt['flux_c1_1'].notna().sum()
    num_radio_Nmips = df_radio_SNRslt[ df_radio_SNRslt[column_radio].notna() & 
                                 df_radio_SNRslt['flux_c1_1'].isna()].shape[0]

    # radio non-detected
    try:
        num_Nradio       = df_Nradio_SNRslt[column_radio].isna().sum()
    except KeyError:
        df_Nradio_SNRslt['ra_radio'] = np.where(df_Nradio_SNRslt['ra_3GHz'].notna(), 
                                       df_Nradio_SNRslt['ra_3GHz'], df_Nradio_SNRslt[column_S1d4GHz])
        num_Nradio       = df_Nradio_SNRslt[column_radio].isna().sum()

    df_Nradio_SNRslt[df_Nradio_SNRslt['flux_3GHz'].isna() & 
                     df_Nradio_SNRslt[column_mips].isna()]
    try:
        num_Nradio_mips  = df_Nradio_SNRslt['ra_nradio_mips'].notna().sum()
    except KeyError:
        df_Nradio_SNRslt['ra_nradio_mips'] = np.where(df_Nradio_SNRslt['flux_3GHz'].isna(), 
                                              df_Nradio_SNRslt[column_mips], np.nan)
    num_Nradio_mips_irac  = df_Nradio_SNRslt['flux_c1_1'].notna().sum()
    num_Nradio_mips_Nirac = df_Nradio_SNRslt[ df_Nradio_SNRslt['ra_nradio_mips'].notna() & 
                                              df_Nradio_SNRslt['flux_c1_1'].isna()].shape[0]
    num_Nradio_Nmips = df_Nradio_SNRslt[ df_Nradio_SNRslt[column_radio].isna() & 
                                        df_Nradio_SNRslt['ra_nradio_mips'].isna()].shape[0]

    num_match_arr = np.array([num_all, num_1d4GHz, num_3GHz, num_1d4GHz_3GHz,
                              num_radio, num_radio_mips, num_radio_Nmips,
                              num_Nradio, num_Nradio_mips, num_Nradio_mips_irac, num_Nradio_mips_Nirac, num_Nradio_Nmips
                             ])
    percent_match_arr = np.around(num_match_arr*100/num_all)

    # print
    num_match_str = np.array2string(num_match_arr, separator=' & ')
    percent_match_str = np.array2string(percent_match_arr, 
                                        formatter={'float_kind':lambda x: "%d"%x},
                                        separator='\% & ')

    print( label_cat +  ' %.1f & '%(snr_thld) +  num_match_str[1:-1] +' \\\\')
    print('& ' + percent_match_str[1:-1] +'\% \\\\')

Gao 3.5 & 472 & 212 & 184 & 150 & 246 & 232 &  14 & 226 & 140 & 123 &  17 &  86 \\
& 100\% & 45\% & 39\% & 32\% & 52\% & 49\% & 3\% & 48\% & 30\% & 26\% &
 4\% & 18\% \\
Gao 4.0 & 354 & 183 & 162 & 135 & 210 & 197 &  13 & 144 &  96 &  84 &  12 &  48 \\
& 100\% & 52\% & 46\% & 38\% & 59\% & 56\% & 4\% & 41\% & 27\% & 24\% &
 3\% & 14\% \\
Gao 4.5 & 279 & 156 & 138 & 120 & 174 & 163 &  11 & 105 &  77 &  67 &  10 &  28 \\
& 100\% & 56\% & 49\% & 43\% & 62\% & 58\% & 4\% & 38\% & 28\% & 24\% &
 4\% & 10\% \\
Gao 5.0 & 234 & 146 & 130 & 114 & 162 & 152 &  10 &  72 &  56 &  47 &   9 &  16 \\
& 100\% & 62\% & 56\% & 49\% & 69\% & 65\% & 4\% & 31\% & 24\% & 20\% &
 4\% & 7\% \\


In [130]:
fn_Gao  = '%scosmos_match_Gao_comb.csv'%(PATH_CAT_CROSSMATCH)
df = pd.read_csv(fn_Gao)

In [131]:
df

,Unnamed: 0,ID_STUDIES_Gao,RA_450Gao,Dec_450Gao,x_450,y_450,S450_raw,S450_corr,S450_err,SNR_450,...,flux_mips24_whw,fluxerr_mips24_whw,SNR_mips24_whw,ra_nradio_mips,dec_nradio_mips,alpha_3_1d4,alpha_3_1d4_err,q_IR,q_IR_err,is_radio_AGNs
0,0,1,150.139038,2.433333,1186,1280,29.612616,28.713087,0.651312,45.466087,...,NaN,NaN,NaN,NaN,NaN,-1.046378,0.150507,2.263253,0.108297,False
1,1,2,150.163503,2.372500,1098,1061,30.387997,29.796791,0.861907,35.256707,...,NaN,NaN,NaN,NaN,NaN,-0.769912,0.099376,2.269797,0.059825,False
2,2,3,150.098725,2.365555,1331,1036,23.406271,22.817402,0.816971,28.650056,...,NaN,NaN,NaN,NaN,NaN,-0.646776,0.263808,2.707072,0.210186,False
3,3,4,150.143207,2.356111,1171,1002,21.652877,21.316823,0.825594,26.227012,...,NaN,NaN,NaN,NaN,NaN,-0.923112,0.068704,1.116226,0.041279,True
4,4,5,150.119853,2.533889,1255,1642,32.558999,31.720827,1.275093,25.534601,...,NaN,NaN,NaN,NaN,NaN,-0.840287,0.119710,2.177413,0.106743,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,467,467,150.221327,2.325553,890,892,8.120222,6.092308,2.317148,3.504404,...,0.4512,0.0174,25.931034,150.221727,2.326323,-0.800000,0.000000,NaN,NaN,False
468,468,468,150.051744,2.306387,1500,823,5.810075,4.504241,1.658105,3.504046,...,0.4311,0.0146,29.527397,150.050830,2.305914,-0.800000,0.000000,NaN,NaN,False
469,469,469,150.081204,2.540832,1394,1667,5.804976,4.498744,1.657561,3.502119,...,NaN,NaN,NaN,NaN,NaN,-0.800000,0.000000,NaN,NaN,False
470,470,470,150.095945,2.364722,1341,1033,2.939532,2.464828,0.839514,3.501470,...,NaN,NaN,NaN,NaN,NaN,-0.800000,0.000000,NaN,NaN,False


In [129]:
# df = df [ df['RA_1d4GHz'].isna() &\
#      df['ra_3GHz'].isna() &\
#      df['id_irac'].isna() &\
# #      df['ID_mips24_LeFloch'].isna()
#      df['RA_mips24_whw'].isna()
#    ]
# df.to_csv('%stest_whw.csv'%(PATH_CAT_CROSSMATCH), index=False, header=True)

In [132]:
df = df[['ID_STUDIES_Gao', 'RA_450Gao', 'Dec_450Gao', 'x_450', 'y_450', 'S450_raw', 'S450_corr', 
    'S450_err', 'SNR_450', 'Comp_450', 'Spur_450', 
    'ID_STUDIES_Lim', 'RA_450Lim', 'DEC_450Lim', 'SCUBA2_450_flux','SCUBA2_450_fluxerr', 
    'SCUBA2_850_flux', 'SCUBA2_850_fluxerr', 'redshift', 'redshift_err', 'redshift_flag',
    'RA_1d4GHz', 'DEC_1d4GHz', 'id_3GHz', 'ra_3GHz', 'dec_3GHz', 
    'id_irac', 'ra_irac', 'dec_irac', 
#     'ID_mips24_LeFloch', 'RA_mips24_LeFloch', 'DEC_mips24_LeFloch'
    'RA_mips24_whw', 'DEC_mips24_whw'
   ]]
if 'Unnamed: 0' in df.columns:
    df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

# fn_out = '%scosmos_Gao_mipsLeFloch_toWH.csv'%(PATH_CAT_CROSSMATCH)
fn_out = '%scosmos_Gao_mipwhwang_toWH.csv'%(PATH_CAT_CROSSMATCH)
df.to_csv(fn_out, index=False, header=True)



In [134]:
df = df [ df['RA_1d4GHz'].isna() &\
     df['ra_3GHz'].isna() &\
     df['id_irac'].isna() &\
#      df['ID_mips24_LeFloch'].isna()
     df['RA_mips24_whw'].isna()
   ]
df = df[['ID_STUDIES_Gao', 'RA_450Gao', 'Dec_450Gao', 'x_450', 'y_450', 'S450_raw', 'S450_corr', 
        'S450_err', 'SNR_450', 'Comp_450', 'Spur_450', 
         'ID_STUDIES_Lim', 'RA_450Lim', 'DEC_450Lim', 'SCUBA2_450_flux','SCUBA2_450_fluxerr', 
         'SCUBA2_850_flux', 'SCUBA2_850_fluxerr', 'redshift', 'redshift_err', 'redshift_flag' 
   ]]
# fn_out2 = '%scosmos_Gao_mipsLeFloch_noID_toWH.csv'%(PATH_CAT_CROSSMATCH)
fn_out2 = '%scosmos_Gao_mipswhwang_noID_toWH.csv'%(PATH_CAT_CROSSMATCH)
df.to_csv(fn_out2, index=False, header=True)



In [121]:
df

,ID_STUDIES_Gao,RA_450Gao,Dec_450Gao,x_450,y_450,S450_raw,S450_corr,S450_err,SNR_450,Comp_450,...,ID_STUDIES_Lim,RA_450Lim,DEC_450Lim,SCUBA2_450_flux,SCUBA2_450_fluxerr,SCUBA2_850_flux,SCUBA2_850_fluxerr,redshift,redshift_err,redshift_flag
252,66,150.100950,2.334722,1323,925,9.624716,8.902381,0.971664,9.905398,0.986931,...,232.0,150.1007,2.3350,6.9163,0.9177,4.5281,0.1492,NaN,NaN,NaN
253,67,150.147657,2.473889,1155,1426,8.165091,7.595560,0.791005,10.322423,0.986743,...,186.0,150.1479,2.4739,5.8308,0.7855,2.8905,0.1442,2.23,0.04,3.0
258,101,150.144320,2.376667,1167,1076,5.906997,5.331160,0.760140,7.770935,0.963902,...,249.0,150.1446,2.3767,4.6220,0.7366,1.7284,0.1235,NaN,NaN,NaN
261,112,150.077039,2.399166,1409,1157,6.358015,5.686786,0.852008,7.462389,0.966155,...,220.0,150.0768,2.3989,4.3528,0.8287,NaN,NaN,NaN,NaN,NaN
265,126,150.174068,2.400555,1060,1162,5.464321,4.779365,0.838592,6.516070,0.918751,...,254.0,150.1735,2.4006,3.8338,0.8197,1.5064,0.1483,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459,456,150.120966,2.493056,1251,1495,3.234412,2.680673,0.914206,3.537947,0.314991,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
462,460,150.110958,2.373055,1287,1063,2.601595,2.247348,0.736363,3.533032,0.323941,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
469,469,150.081204,2.540832,1394,1667,5.804976,4.498744,1.657561,3.502119,0.302216,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
470,470,150.095945,2.364722,1341,1033,2.939532,2.464828,0.839514,3.501470,0.322971,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
